In [20]:
import numpy as np
import pandas as pd
import requests
from sklearn.metrics import roc_curve, accuracy_score
from sklearn.model_selection import StratifiedKFold

train = pd.read_csv('EdTech_train_data.csv')

X = train.drop(['No','Rank'],axis=1).values
y = train['Rank'].values


In [22]:
train.eval()
train.head()


,No,Education,Major,Year,TargetMajor,Degree Application,GPA,GMAT,IBT,GRE,IETLS,Distinguished Internship,General Internship,Research Experience,Other Experience,Rank
0,187,3,237,2010,171,1,3.70,0.0,102.0,327.500000,7.0,4,2,6,0,3
1,271,3,159,2009,349,1,3.38,0.0,97.0,272.211083,5.5,2,2,5,0,2
2,790,3,273,2011,293,3,3.40,0.0,101.0,325.000000,7.0,4,2,6,5,3
3,579,3,299,2008,372,1,3.80,0.0,0.0,0.000000,0.0,1,0,1,0,0
4,479,3,224,2009,275,2,3.27,0.0,88.0,320.000000,6.0,2,0,5,0,2


In [2]:
train.shape

(900, 16)

In [3]:
from sklearn import preprocessing
# standardize the data attributes
standardized_X = preprocessing.scale(X)
# normalize the data attributes
normalized_X = preprocessing.normalize(X)

In [4]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=0)
accs = []
for train_index, test_index in skf.split(standardized_X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test,y_pred)
    accs.append(acc)
    print('accuracy:', acc)
print('mean_acc',np.mean(np.array(accs)))

accuracy: 0.8681318681318682
accuracy: 0.8131868131868132
accuracy: 0.8571428571428571
accuracy: 0.8222222222222222
accuracy: 0.9
accuracy: 0.9222222222222223
accuracy: 0.8777777777777778
accuracy: 0.8666666666666667
accuracy: 0.8539325842696629
accuracy: 0.8863636363636364
mean_acc 0.8667646647983727


In [5]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model,14)
rfe = rfe.fit(X, y)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [6]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X, y)
print(model)
# make predictions
expected = y
predicted = model.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

# use cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=5)
print (scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0       0.70      0.46      0.55       160
          1       0.71      0.77      0.74       239
          2       0.89      0.98      0.93       333
          3       0.97      1.00      0.99       168

avg / total       0.82      0.83      0.82       900

[[ 73  72  15   0]
 [ 29 183  27   0]
 [  2   1 325   5]
 [  0   0   0 168]]
[0.80662983 0.83425414 0.81767956 0.81005587 0.83146067]
Accuracy: 0.82 (+/- 0.02)


In [7]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X, y)
print(model)
# make predictions
expected = y
predicted = model.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
# print(metrics.confusion_matrix(expected, predicted))

# use cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=5)
print (scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

GaussianNB(priors=None)
             precision    recall  f1-score   support

          0       0.76      0.29      0.42       160
          1       0.66      0.90      0.76       239
          2       0.95      0.98      0.97       333
          3       1.00      0.98      0.99       168

avg / total       0.85      0.84      0.82       900

[0.83977901 0.77348066 0.85082873 0.83240223 0.81460674]
Accuracy: 0.82 (+/- 0.05)


In [8]:
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
# fit a k-nearest neighbor model to the data
model = KNeighborsClassifier()
model.fit(X, y)
print(model)
# make predictions
expected = y
predicted = model.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

# use cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=5)
print (scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
             precision    recall  f1-score   support

          0       0.56      0.59      0.57       160
          1       0.53      0.58      0.56       239
          2       0.60      0.68      0.64       333
          3       0.72      0.42      0.53       168

avg / total       0.60      0.59      0.58       900

[[ 94  33  27   6]
 [ 32 139  58  10]
 [ 32  65 225  11]
 [ 10  23  65  70]]
[0.46961326 0.38121547 0.35911602 0.33519553 0.43258427]
Accuracy: 0.40 (+/- 0.10)


In [9]:
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
# fit a CART model to the data
model = DecisionTreeClassifier()


#model.fit(X, y)
'''print(model)
# make predictions
expected = y
predicted = model.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
'''

# use cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=5)
print (scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.85635359 0.86740331 0.83977901 0.82681564 0.86516854]
Accuracy: 0.85 (+/- 0.03)


In [10]:
from sklearn import metrics
from sklearn.cluster import KMeans
# fit a SVM model to the data
model = KMeans()
'''model.fit(X, y)
print(model)
# make predictions
expected = y
predicted = model.predict(X)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))
'''
# use cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=10)
print (scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[-1592495.48292073 -1830665.90843967 -1800987.38371087 -2126852.0348177
 -1790606.6070179  -1866615.9778919  -1348854.93343377 -1898973.65282763
 -1842601.15305884 -1679324.47637093]
Accuracy: -1777797.76 (+/- 389940.19)


In [11]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
# prepare a range of alpha values to test
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
# create and fit a ridge regression model, testing each alpha
model = Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))
grid.fit(X, y)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha)

# use cross validation
from sklearn.model_selection import cross_val_score
scores = cross_val_score(grid, X, y, cv=5)
print (scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

GridSearchCV(cv=None, error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 0.e+00])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
0.8288717651309263
1.0
[0.80370133 0.80374185 0.82941926 0.84673366 0.86400241]
Accuracy: 0.83 (+/- 0.05)


In [12]:
import numpy as np
from scipy.stats import uniform as sp_rand
from sklearn.linear_model import Ridge
from sklearn.model_selection import RandomizedSearchCV
# prepare a uniform distribution to sample for the alpha parameter
param_grid = {'alpha': sp_rand()}
# create and fit a ridge regression model, testing random alpha values
model = Ridge()
rsearch = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=100)
rsearch.fit(X, y)
print(rsearch)
# summarize the results of the random parameter search
print(rsearch.best_score_)
print(rsearch.best_estimator_.alpha)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(rsearch, X, y, cv=5)
print (scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
          fit_params=None, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000210BA5BA6A0>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)
0.8288717399094397
0.9956005524208634
[0.80370179 0.80374076 0.82941926 0.84673368 0.86400232]
Accuracy: 0.83 (+/- 0.05)


In [13]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = DecisionTreeClassifier()
#odel.fit(X,y)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv = 5)
print (np.mean(scores))

0.8466717831144045


In [14]:
skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=0)
accs = []
model = DecisionTreeClassifier()
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test,y_pred)
    accs.append(acc)
    print('accuracy:', acc)
print('mean_acc',np.mean(np.array(accs)))

accuracy: 0.8021978021978022
accuracy: 0.8681318681318682
accuracy: 0.8241758241758241
accuracy: 0.8555555555555555
accuracy: 0.8777777777777778
accuracy: 0.8444444444444444
accuracy: 0.8
accuracy: 0.8333333333333334
accuracy: 0.8089887640449438
accuracy: 0.875
mean_acc 0.8389605369661549


In [15]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [16]:
skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=0)
accs = []
model = xgb.XGBClassifier(subsample=0.5,nthread=7,objective='muti:softmax',num_class=1)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test,y_pred)
    accs.append(acc)
    print('accuracy:', acc)
print('mean_acc',np.mean(np.array(accs)))
accs

C:\Users\44994\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\44994\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


accuracy: 0.9120879120879121
accuracy: 0.9120879120879121


C:\Users\44994\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\44994\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


accuracy: 0.8461538461538461
accuracy: 0.8555555555555555


C:\Users\44994\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\44994\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


accuracy: 0.9
accuracy: 0.9111111111111111


C:\Users\44994\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\44994\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


accuracy: 0.8333333333333334
accuracy: 0.8111111111111111
accuracy: 0.898876404494382
accuracy: 0.8977272727272727
mean_acc 0.8778044458662437


C:\Users\44994\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\44994\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[0.9120879120879121,
 0.9120879120879121,
 0.8461538461538461,
 0.8555555555555555,
 0.9,
 0.9111111111111111,
 0.8333333333333334,
 0.8111111111111111,
 0.898876404494382,
 0.8977272727272727]

In [17]:
# y_pred = model.predict(X_test)
# predictions = [round(value) for value in y_pred]

In [18]:
# accuracy = accuracy_score(y_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))
# from sklearn.model_selection import cross_val_score
# scores = cross_val_score(model, X, y, cv=5)
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [19]:
from sklearn.ensemble import RandomForestClassifier
skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=0)
accs = []
model = RandomForestClassifier(n_estimators=100, random_state=2, max_depth=100)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test,y_pred)
    accs.append(acc)
    print('accuracy:', acc)
print('mean_acc',np.mean(np.array(accs)))

accuracy: 0.9120879120879121
accuracy: 0.8791208791208791
accuracy: 0.8681318681318682
accuracy: 0.8888888888888888
accuracy: 0.9111111111111111
accuracy: 0.9111111111111111
accuracy: 0.8555555555555555
accuracy: 0.8666666666666667
accuracy: 0.9101123595505618
accuracy: 0.9090909090909091
mean_acc 0.8911877261315462
